<a href="https://colab.research.google.com/github/Loptt/forest-fire-detector/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fire Dectection Classifier

## Image Preprocessing and Saving

In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import queue
import logging
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, color
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from PIL import Image
%matplotlib inline

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

# path = "/content/gdrive/My Drive/SWE/Original_Data/Train/" # Colab
path = "data/Training/" # Local

train_fire_imgs = []

for i in range(30000): #30000
    try:
        train_fire_imgs.append(io.imread(path + "Fire/resized_frame{}.jpg".format(i)))
        print(f'Loaded {i}')
    except:
        # No image found
        pass

25890
Loaded 25891
Loaded 25892
Loaded 25893
Loaded 25894
Loaded 25895
Loaded 25896
Loaded 25897
Loaded 25898
Loaded 25899
Loaded 25900
Loaded 25901
Loaded 25902
Loaded 25903
Loaded 25904
Loaded 25905
Loaded 25906
Loaded 25907
Loaded 25908
Loaded 25909
Loaded 25910
Loaded 25911
Loaded 25912
Loaded 25913
Loaded 25914
Loaded 25915
Loaded 25916
Loaded 25917
Loaded 25918
Loaded 25919
Loaded 25920
Loaded 25921
Loaded 25922
Loaded 25923
Loaded 25924
Loaded 25925
Loaded 25926
Loaded 25927
Loaded 25928
Loaded 25929
Loaded 25930
Loaded 25931
Loaded 25932
Loaded 25933
Loaded 25934
Loaded 25935
Loaded 25936
Loaded 25937
Loaded 25938
Loaded 25939
Loaded 25940
Loaded 25941
Loaded 25942
Loaded 25943
Loaded 25944
Loaded 25945
Loaded 25946
Loaded 25947
Loaded 25948
Loaded 25949
Loaded 25950
Loaded 25951
Loaded 25952
Loaded 25953
Loaded 25954
Loaded 25955
Loaded 25956
Loaded 25957
Loaded 25958
Loaded 25959
Loaded 25960
Loaded 25961
Loaded 25962
Loaded 25963
Loaded 25964
Loaded 25965
Loaded 25966
Loaded

In [ ]:
train_no_fire_imgs = []

for i in range(30000): #30000
    try:
        train_no_fire_imgs.append(io.imread(path + "No_Fire/lake_resized_lake_frame{}.jpg".format(i)))
    except:
        # No image found
        pass

for i in range(30000): #30000
    try:
        train_no_fire_imgs.append(io.imread(path + "No_Fire/resized_frame{}.jpg".format(i)))
    except:
        # No image found
        pass

In [ ]:
def calculate_mean_channel(img_lab, n):
    channel = img_lab[:,:,n].flatten()
    return np.mean(channel)

def calculate_rx(img_lab, n, m):
    r1 = img_lab[:,:,n] >= m
    return r1 

def clean_small(mat, threshold):
    visited = np.full(mat.shape, False)
    to_remove = []
    cleaned_mat = np.copy(mat)
    q = queue.Queue()
    size = 0

    for i in range(len(mat)):
        for j in range(len(mat[0])):
            if not mat[i,j]:
                continue
            # For each True pixel perfom BFS and calculate size of cluster
            size = 0
            to_remove = []
            q.put((i,j))
            while not q.empty():
                size += 1
                x, y = q.get()
                if (x < len(mat) and x >= 0 and y < len(mat[0]) and y >= 0) and not visited[x,y] and mat[x,y]:
                    visited[x,y] = True
                    to_remove.append((x,y))
                    q.put((x+1, y))
                    q.put((x-1, y))
                    q.put((x, y+1))
                    q.put((x, y-1))
            # If size is smaller than predefined threshold, remove nodes in to_remove
            # print("Report: Size: {} i: {}, j {}".format(size, i, j))
            if size < threshold:
                for x, y in to_remove:
                    cleaned_mat[x,y] = False
    return cleaned_mat

def apply_mask(img, mask):
    masked = np.copy(img)
    for i in range(len(masked)):
        for j in range(len(masked[0])):
            masked[i,j] = masked[i,j] if mask[i,j] else [0,0,0]
    
    return masked

def enhance_brightness(img, am):
    enhanced = np.copy(img)

    if am < 0:
        return enhanced

    for i in range(len(enhanced)):
        for j in range(len(enhanced[0])):
            for c in range(len(enhanced[0,0])):
                enhanced[i,j,c] = (int(enhanced[i,j,c]) * am) if (int(enhanced[i,j,c]) * am) < 256 else 255
    return enhanced 

def full_transform(img_rgb, enhance, threshold, msg=""):
    img_enc = enhance_brightness(img_rgb, enhance)
    img_lab = color.rgb2lab(img_enc)
    l_m = calculate_mean_channel(img_lab, 0)
    a_m = calculate_mean_channel(img_lab, 1)
    b_m = calculate_mean_channel(img_lab, 2)

    r1 = calculate_rx(img_lab, 0, l_m)
    r2 = calculate_rx(img_lab, 1, a_m)
    r3 = calculate_rx(img_lab, 2, b_m)
    r4 = calculate_rx(img_lab, 2, img_lab[:,:,1])

    r_and = np.logical_and(np.logical_and(np.logical_and(r1, r2), r3), r4)
    r_cleaned = clean_small(r_and, threshold)

    new_img = apply_mask(img_rgb, r_cleaned)

    logging.debug("Transofrmed image " + msg)

    return apply_mask(img_rgb, r_cleaned)

In [ ]:
enhance = 1
threshold = 15

print("Total fire:", len(train_fire_imgs))

train_fire_imgs_transformed = [full_transform(img, enhance, threshold, "Fire {}".format(i)) for i, img in enumerate(train_fire_imgs)]

for i, img in enumerate(train_fire_imgs_transformed):
    img = Image.fromarray(img)
    img.save("data_transformed/Training/Fire/frame_" + str(i) + ".jpg")


DEBUG:root:Transofrmed image Fire 24518
DEBUG:root:Transofrmed image Fire 24519
DEBUG:root:Transofrmed image Fire 24520
DEBUG:root:Transofrmed image Fire 24521
DEBUG:root:Transofrmed image Fire 24522
DEBUG:root:Transofrmed image Fire 24523
DEBUG:root:Transofrmed image Fire 24524
DEBUG:root:Transofrmed image Fire 24525
DEBUG:root:Transofrmed image Fire 24526
DEBUG:root:Transofrmed image Fire 24527
DEBUG:root:Transofrmed image Fire 24528
DEBUG:root:Transofrmed image Fire 24529
DEBUG:root:Transofrmed image Fire 24530
DEBUG:root:Transofrmed image Fire 24531
DEBUG:root:Transofrmed image Fire 24532
DEBUG:root:Transofrmed image Fire 24533
DEBUG:root:Transofrmed image Fire 24534
DEBUG:root:Transofrmed image Fire 24535
DEBUG:root:Transofrmed image Fire 24536
DEBUG:root:Transofrmed image Fire 24537
DEBUG:root:Transofrmed image Fire 24538
DEBUG:root:Transofrmed image Fire 24539
DEBUG:root:Transofrmed image Fire 24540
DEBUG:root:Transofrmed image Fire 24541
DEBUG:root:Transofrmed image Fire 24542


In [ ]:
enhance = 1
threshold = 20

print("Total No fire:", len(train_no_fire_imgs))

train_no_fire_imgs_transformed = [full_transform(img, enhance, threshold, "No Fire {}".format(i)) for i, img in enumerate(train_no_fire_imgs)]

for i, img in enumerate(train_no_fire_imgs_transformed):
    img = Image.fromarray(img)
    img.save("data_transformed/Training/No_Fire/frame_" + str(i) + ".jpg")

3891
DEBUG:root:Transofrmed image No Fire 13892
DEBUG:root:Transofrmed image No Fire 13893
DEBUG:root:Transofrmed image No Fire 13894
DEBUG:root:Transofrmed image No Fire 13895
DEBUG:root:Transofrmed image No Fire 13896
DEBUG:root:Transofrmed image No Fire 13897
DEBUG:root:Transofrmed image No Fire 13898
DEBUG:root:Transofrmed image No Fire 13899
DEBUG:root:Transofrmed image No Fire 13900
DEBUG:root:Transofrmed image No Fire 13901
DEBUG:root:Transofrmed image No Fire 13902
DEBUG:root:Transofrmed image No Fire 13903
DEBUG:root:Transofrmed image No Fire 13904
DEBUG:root:Transofrmed image No Fire 13905
DEBUG:root:Transofrmed image No Fire 13906
DEBUG:root:Transofrmed image No Fire 13907
DEBUG:root:Transofrmed image No Fire 13908
DEBUG:root:Transofrmed image No Fire 13909
DEBUG:root:Transofrmed image No Fire 13910
DEBUG:root:Transofrmed image No Fire 13911
DEBUG:root:Transofrmed image No Fire 13912
DEBUG:root:Transofrmed image No Fire 13913
DEBUG:root:Transofrmed image No Fire 13914
DEBUG:

## Loading Preprocessed images
If there are already preprocessed images in data_transformed/ load this to skip the image preprocessing which can take hours.
We are also saving the Fire and No_Fire into a single array with 0 denoting No Fire and 1 denoting Fire.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

data = []

# path = "data_transformed/Training/Fire/" # Local
path = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Transformed/Training/" # Colab

In [ ]:
for i in range(30000):
    try:
        data.append((np.uint8(io.imread(path + "Fire/frame_{}.jpg".format(i))), 1))
        logging.debug(f"Loading Fire Transformed {i}")
    except:
        # No image found
        pass

In [ ]:
for i in range(30000):
    try:
        data.append((np.uint8(io.imread(path + "No_Fire/frame_{}.jpg".format(i))), 0))
        logging.debug(f"Loading No Fire Transformed {i}")
    except:
        # No image found
        pass

In [ ]:
random.shuffle(data)

train_x = np.array([point[0] for point in data])
train_y = np.array([point[1] for point in data])

print(train_x.shape)
print(train_y.shape)
print(type(train_x))

(20967, 254, 254, 3)
(20967,)


In [ ]:
clf = SVC(kernel="rbf", C=1.0)
clf.fit(train_x, train_y)




MemoryError: Unable to allocate 30.2 GiB for an array with shape (20967, 254, 254, 3) and data type float64